In [26]:
from utils import load_data, check_gpu, check_dir
from clean_session import preprocess_sessions
import pandas as pd
import numpy as np
from tqdm import tqdm
import gc
from functools import partial
import matplotlib.pyplot as plt
from clean_session import preprocess_sessions
from create_model_inputs import prepare_data, compute_session_fts, create_model_inputs, meta_encoding
import datetime
%matplotlib inline
%load_ext autoreload
%autoreload 2
def fprint(df, name):
    print(f'{name} shape: ({df.shape[0]:,}, {df.shape[1]})')
import pprint
pp = pprint.PrettyPrinter(indent=1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
nrows = 1000000
train = prepare_data('train', nrows=nrows, recompute=False)
df = compute_session_fts(train, 'train')

[05-25 12:00:56 - create_model_inputs-48 - prepare_data - INFO] Load from existing ./nn_cache/train_1000000_no_test.snappy
[05-25 12:01:33 - create_model_inputs-158 - compute_session_fts - INFO] Session features generated: ['timestamp_session_duration', 'timestamp_dwell_time_prior_clickout', 'current_filters_last_filters', 'session_id_size']


In [6]:
df = df.groupby('session_id').last().reset_index()
# df['session_id_size'] = np.log(df['session_id_size'])
# df['cfs'] = df['current_filters_last_filters'].str.lower().str.split('|')

# df['prices'] = df['prices'].str.split('|')
# df['prices'] = df['prices'].apply(lambda x: [int(p) for p in x])

# df['time_steps'] = df['prices'].str.len()
# padding_mask = df['time_steps'] < 25
# df.loc[padding_mask, 'prices'] = df.loc[padding_mask, 'prices'].apply(lambda x: np.pad(x, (0, 25-len(x)),
#                                                                                        mode='constant'))
df['imps'] = df['impressions'].str.split('|')
df['imps'] = df['imps'].apply(lambda x: [int(i) for i in x])
# df.loc[padding_mask, 'impressions'] = (df.loc[padding_mask, 'impressions']
#                                          .apply(lambda x: np.pad(x, (0, 25-len(x)),mode='constant')))
df['reference'] = df['reference'].astype(int)
# filter out nan rows with reference_id not in impressions list, since if the true target in test
# is not in the impression list then it would not get evaluated
def assign_target(row):
    ref = row['reference']
    imp = list(row['imps'])
    if ref in imp:
        return imp.index(ref)
    else:
        return np.nan

df['target'] = df.apply(assign_target, axis=1)
# drop the ones whose reference is not in the impression list
df = df[df['target'].notna()].reset_index(drop=True)
df['target'] = df['target'].astype(int)

### comapre session with size 1 and the rest

In [7]:
sids = df['session_id'].unique()

---
### ones

In [8]:
display_cols = ['session_id', 'step', 'action_type', 'current_filters', 'reference', 'timestamp',
                'impressions', 'prices', 'target', 'session_id_size']
raw_display_cols = [c for c in train.columns if c in display_cols]

In [9]:
# pd.options.display.max_colwidth = 100

In [78]:
rid = np.random.choice(sids, 1)[0]
print(rid)
df[df.session_id==rid][display_cols].reset_index(drop=True)

fedf037948a86


,session_id,step,action_type,current_filters,reference,timestamp,impressions,prices,target,session_id_size
0,fedf037948a86,3,clickout item,None,1656117,2018-11-05 13:50:38,942193|50161|49151|10321480|394531|1256004|108...,38|84|62|57|50|37|130|59|47|64|45|95|39|38|64|...,8,2


In [79]:
# pp.pprint(df[df.session_id==rid][display_cols].reset_index(drop=True).to_dict(orient='list'))

In [80]:
# and the raw
raw_train_demo = train[train.session_id==rid][raw_display_cols].reset_index(drop=True)
raw_train_demo

,session_id,timestamp,step,action_type,current_filters,reference,impressions,prices
0,fedf037948a86,2018-11-05 13:50:33,2,interaction item image,None,1656117,None,None
1,fedf037948a86,2018-11-05 13:50:38,3,clickout item,None,1656117,942193|50161|49151|10321480|394531|1256004|108...,38|84|62|57|50|37|130|59|47|64|45|95|39|38|64|...


In [63]:
# pp.pprint(train[train.session_id==rid][raw_display_cols].reset_index(drop=True).to_dict(orient='list'))

---
### the generated train input

In [64]:
# !ls -lthr ./nn_cache/

In [17]:
tids = np.load('./nn_cache/train_session_ids.npy')
merged = np.load('./nn_cache/train_merged.npy')

In [81]:
mask = tids==rid

In [66]:
merged[mask].shape

(1, 25, 159)

In [91]:
look_imp_ind = 2
merged[mask][0][look_imp_ind]

array([0.        , 0.        , 0.        , 1.        , 0.        ,
       1.        , 0.        , 0.        , 0.        , 1.        ,
       0.        , 1.        , 1.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 1.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 1.        , 1.        , 0.        ,
       0.        , 0.        , 0.        , 1.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 1.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 1.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 1.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 1.        , 0.        , 0.     

In [83]:
raw_train_demo

,session_id,timestamp,step,action_type,current_filters,reference,impressions,prices
0,fedf037948a86,2018-11-05 13:50:33,2,interaction item image,None,1656117,None,None
1,fedf037948a86,2018-11-05 13:50:38,3,clickout item,None,1656117,942193|50161|49151|10321480|394531|1256004|108...,38|84|62|57|50|37|130|59|47|64|45|95|39|38|64|...


In [85]:
raw_demo_prices = raw_train_demo.prices.str.split('|').iloc[1]
raw_demo_prices = [int(i) for i in raw_demo_prices]

In [86]:
raw_demo_prices

[38, 84, 62, 57, 50, 37, 130, 59, 47, 64, 45, 95, 39, 38, 64, 58, 39, 70]

In [87]:
from scipy.stats import rankdata
ranks = rankdata(raw_demo_prices, method='dense')
ranks

array([ 2, 13, 10,  7,  6,  1, 15,  9,  5, 11,  4, 14,  3,  2, 11,  8,  3,
       12])

In [92]:
ranks[look_imp_ind]/np.max(ranks)

0.6666666666666666

In [93]:
raw_demo_prices[look_imp_ind]/np.max(raw_demo_prices)

0.47692307692307695

In [31]:
meta_mapping, _ = meta_encoding()

[05-25 13:39:42 - create_model_inputs-179 - meta_encoding - INFO] Load from existing file: ./nn_cache/meta_mapping.npy


In [94]:
meta_mapping[49151]

array([0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0,
       0, 1, 1])

In [29]:
meta_mapping

({5101: array([0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
         0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0,
         1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0,
         0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1,
         0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0,
         1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,
         0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0,
         0, 1, 0]),
  5416: array([0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
         0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
         1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0,
         0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0,
         1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,

In [95]:
numerics = np.load('./nn_cache/train_numerics.npy')

In [105]:
np.mean(numerics[:, [0,1]], axis=0)

array([1.57020513, 2.60355986])

In [104]:
numerics[:, [0,1]] - np.mean(numerics[:, [0,1]], axis=0)

array([[ 0.22155434,  2.86450028],
       [-0.47159284, -1.91041268],
       [ 0.50923641,  1.22508154],
       ...,
       [ 0.73237996,  0.53193436],
       [ 0.50923641,  0.03549747],
       [ 0.37570502,  0.03549747]])

In [106]:
numerics[:, [0,1]]

array([[1.79175947, 5.46806014],
       [1.09861229, 0.69314718],
       [2.07944154, 3.8286414 ],
       ...,
       [2.30258509, 3.13549422],
       [2.07944154, 2.63905733],
       [1.94591015, 2.63905733]])

In [107]:
1.79175947 - 0.22155434

1.57020513

In [108]:
2.86450028 - 5.46806014

-2.6035598600000003